# LLM with Web Search and Crawl

풍부한 컨텍스틀 활용하기 위해 구글검색 된 결과의 최상위 n개의 페이지를 크롤링하여 LLM에 제공하기 위한 코드입니다.



In [61]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True)

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

bs4 or scrapy?

In [ ]:
import requests
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

def extract_text_and_tables_by_bs4(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    text = "\n".join(paragraphs)
    return text


async def extract_text_and_tables_async(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    async with httpx.AsyncClient(timeout=2) as client:
        try:
            response = await client.get(url, headers=headers)
            response.raise_for_status()
        except httpx.HTTPError as e:
            print(f"Request failed: {e}")
            return ""

        selector = scrapy.Selector(text=response.text)
        paragraphs = [p.strip() for p in selector.css('p::text').getall() if p.strip()]
        text = "\n".join(paragraphs)
        return text

async def add_context_async(top_urls = []):
    async def gather_contexts():
        tasks = [extract_text_and_tables_async(url) for url in top_urls]
        results = await asyncio.gather(*tasks)
        return results
    return await gather_contexts()

def google_search(query, num=5):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
        "siteSearch": "samsung.com",
        "siteSearchFilter": "e",
    }
    response = requests.get(url, params=params)
    results = response.json()
    return results.get("items", [])

       
QUERY_REWRITE_PROMPT = """
        너는 구글 검색과 LLM을 위한 질문 재작성 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해줘.

        Google Search용 Query Rewrite 프롬프트
                사용자의 질문을 실제 구글 검색창에 입력할 수 있도록, 명확하고 간결한 검색어로 재작성해줘. 불필요한 문장이나 맥락 설명은 빼고, 핵심 키워드와 관련 정보를 중심으로 검색에 최적화된 형태로 만들어줘.
        LLM Query용 Rewrite 프롬프트
                사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해줘. 필요한 경우, 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어줘.

        예시:   
        * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
        * 구글 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
        * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

        출력포맷. 반드시 json 형식으로 출력해줘.:
        {"google_search": "구글 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())







In [79]:
from IPython.display import Markdown, display
from datetime import datetime

RESULTS_COUNT = 3

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

# inputs = ["삼성전자 25년 제품이 작년 대비 좋아진것은"]

for input in inputs:
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Google Search Query: {query_rewrite['google_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    results = google_search(query_rewrite['google_search'], RESULTS_COUNT)
    print(f"Google Search Results: {len(results)}")
    top_urls = [results[i]["link"] for i in range(len(results))]

    contexts = await add_context_async(top_urls)

    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context[:1000]}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    system_prompt = "너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘."
    user_prompt = f"""
        너는 아래 제공하는 구글에서 검색한 컨텍스트를 바탕으로 질문에 대한 답변을 제공해야 해. 컨텍스트를 최대한 활용하여 풍부하게 답변을 해야해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.

        구글에서 제공한 컨텍스트: {contexts}
        질문: {query_rewrite['llm_query']}
        """

    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))

Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Google Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 중 2구 모델이 아닌 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 설명해 주세요.
Google Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 중 2구 모델이 아닌 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 설명해 주세요.
Google Search Results: 3
Google Search Results: 3


아래는 2025년 5월 기준, 삼성전자 3구 이상 인덕션 제품 중 추천할 만한 모델과 주요 기능 및 장점입니다.

---

## 1. 삼성 비스포크 AI 3구 인덕션 NZ63DB607CF
- **화력**  
  - 대화구 3,400W / 중화구 2,600W / 소화구 1,200W로 상위 10% 내 최상급 화력 보유  
- **주요 기능 및 장점**  
  - AI 넘침 방지 모드 탑재로 조리 시 편리함 및 안전 강화  
  - 저소음 기능(DCN) 적용  
  - 대화형 알림창과 심플해진 1개 조작부로 편의성 향상  
  - 프레임리스 디자인과 얇은 두께로 주방 인테리어 미려, 식기세척기와 설치 시 공간 활용 우수  
- **가격대**  
  - 100만원 미만으로 기능 대비 뛰어난 가성비 제공  
- **상판**  
  - NEG 제품 상판 (SCHOTT 제품보다 원가 절감, 체감 효용 미미)

---

## 2. 삼성 비스포크 AI 3구 인덕션 NZ63DB607CABS (상위 모델)
- **화력**  
  - NZ63DB607CF와 동일  
- **주요 기능 및 장점**  
  - 상판으로 업계 최고급 SCHOTT 제품 사용  
  - 스마트컨트롤 앱 연동 가능(외부에서 전원 제어)  
  - 전반적인 기능과 품질에서 한 단계 상위  
- **가격대**  
  - 96~115만원 수준, 픽 제품 대비 근소한 가격 차이

---

## 3. 삼성 비스포크 3구 인덕션 NZ63D650BXE (하위 모델)
- **화력**  
  - 픽 제품과 동일한 출력  
- **주요 기능 및 장점**  
  - AI 끓음 감지 기능 미탑재  
  - 조작부 각 화구별 3개 탑재 (픽 제품과 조작 방식 차이)  
  - 상대적으로 두꺼운 두께  
- **가격대**  
  - 66~79만원, 예산 절약용으로 적합

---

## 4. 삼성 3구 인덕션 NZ63B4026AK (가성비 모델)
- **화력**  
  - 픽 제품과 동일한 출력  
- **주요 기능 및 장점**  
  - 상판은 블랙 컬러, 대화형 알림창 미탑재  
  - 편의 기능은 축소되었지만 가격이 매우 저렴  
- **가격대**  
  - 55~65만원, 가성비 중시 사용자에게 추천

---

## 5. 삼성 하이브리드 인덕션 NZ63T4507AK (인덕션 + 하이라이트 조합)
- **화력**  
  - 인덕션 대화구 2,300W  
  - 중화구 2개는 하이라이트 방식  
- **주요 기능 및 장점**  
  - 인덕션과 하이라이트 조합으로 다양한 용기 사용 가능  
  - 하이라이트 중화구 직경 20cm로 넓은 용기 대응 가능  
  - SCHOTT 상판 탑재로 내구성 우수  
- **가격대**  
  - 110~132만원, 인덕션과 하이라이트의 장점을 모두 원하는 사용자에 적합

---

## 요약

| 모델명                | 화구수 | 주요 화력(W)                  | 주요 특징                             | 가격대(만원)   |
|---------------------|-------|-----------------------------|---------------------------------|--------------|
| NZ63DB607CF         | 3구    | 3,400 / 2,600 / 1,200       | AI 넘침방지, 저소음, 프레임리스, 가성비 우수  | 80~100       |
| NZ63DB607CABS       | 3구    | 3,400 / 2,600 / 1,200       | SCHOTT 상판, 스마트컨트롤          | 96~115       |
| NZ63D650BXE         | 3구    | 3,400 / 2,600 / 1,200       | 기본형, AI 기능 미탑재             | 66~79        |
| NZ63B4026AK         | 3구    | 3,400 / 2,600 / 1,200       | 가성비 특화, 알림창 없음            | 55~65        |
| NZ63T4507AK (하이브리드) | 3구    | 인덕션 2,300 + 하이라이트 2개 | 하이브리드, 다양한 용기 사용 가능    | 110~132      |

---

필요하신 용도와 예산에 따라 위 제품 중에서 선택하시면 됩니다.  
- **최상급 화력과 편의 기능, 디자인을 원한다면 NZ63DB607CF 추천**  
- **더 좋은 상판과 스마트 기능을 원하면 NZ63DB607CABS**  
- **가성비 우선이면 NZ63B4026AK 또는 NZ63D650BXE 고려**  
- **하이라이트 병행 사용 희망 시 하이브리드 NZ63T4507AK 적합**

궁금한 점 있으면 추가로 문의해 주세요!

Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Google Search Query: 삼성전자 TV 추천 부모님 선물
LLM Query: 부모님께 선물하기 좋은 삼성전자 TV를 추천해 주세요. 사용 용도와 예산에 맞는 모델과 주요 기능, 장점을 함께 알려주시면 좋겠습니다.
Google Search Query: 삼성전자 TV 추천 부모님 선물
LLM Query: 부모님께 선물하기 좋은 삼성전자 TV를 추천해 주세요. 사용 용도와 예산에 맞는 모델과 주요 기능, 장점을 함께 알려주시면 좋겠습니다.
Google Search Results: 3
Google Search Results: 3
Request failed: 
Request failed: 


안녕하세요! 2025년 5월 현재, 부모님께 선물하기 좋은 삼성전자 TV 추천을 도와드리겠습니다. 제공해주신 컨텍스트와 최신 정보를 바탕으로 답변드립니다.

---

## 1. 예산 및 사용 용도 고려

- **예산:** 약 130만원
- **크기:** 75인치 정도 (거실용, 큰 화면 선호)
- **용도:** 자연 다큐, 여행기 등 영상 시청이 주 목적이며 화질이 중요

---

## 2. 추천 모델

### 삼성전자 QLED 75Q60C (2024년형)

| 항목 | 내용 |
|---|---|
| **화면 크기** | 75인치 |
| **패널 타입** | QLED (퀀텀닷 디스플레이) |
| **해상도** | 4K UHD (3840x2160) |
| **HDR 지원** | HDR10+, HDR10, HLG |
| **스마트 기능** | 삼성 스마트 허브, 다양한 OTT 앱 지원, 음성인식 (Bixby, Alexa, Google Assistant) |
| **주요 기능** | AI 업스케일링, 자동 장면 최적화, 에코 세이빙, 다중 음성 지원 |
| **가격대** | 약 120~130만원 (할인 행사 시 가격 변동 가능) |

#### 주요 장점
- **화질:** 삼성 QLED 특유의 밝고 선명한 색감과 깊은 명암비로 자연 다큐, 여행기 영상에 적합
- **브랜드 신뢰성:** 국내 대기업 삼성전자 제품으로 A/S 및 고객 서비스 우수
- **사용 편의성:** 직관적인 스마트 인터페이스와 음성 명령 지원으로 부모님도 쉽게 사용 가능
- **내구성:** 긴 수명과 안정적인 성능 제공

---

## 3. 참고 사항 및 비교

- TCL 제품 대비 삼성 QLED는 브랜드 신뢰도와 A/S 측면에서 우수하므로 부모님 세대에 더욱 적합합니다.
- LED 대비 QLED는 더 좋은 화질과 색 재현력을 보여주어 영상 감상에 더욱 만족도가 높습니다.
- 75인치 이상 대형 TV는 거실 크기와 시청 거리 고려 필수입니다.

---

## 4. 구매 팁

- 쿠팡, 삼성 공식몰, 주요 전자제품 전문점에서 할인 행사(명절, 연말 등)를 노리면 예산 내에 더 좋은 모델을 구입 가능
- 삼성 QLED 60~70 시리즈도 예산 범위 내에서 좋은 선택지이며, 75인치 모델은 할인 시기를 잘 활용하는 것이 좋음

---

## 정리

| 추천 모델 | 삼성 QLED 75Q60C (2024년형) |
|---|---|
| 주요 특징 | 75인치 대형 QLED, 4K, HDR10+, 스마트 기능 |
| 가격대 | 약 120~130만원 |
| 장점 | 뛰어난 화질, 브랜드 신뢰성, 편리한 사용성, 우수한 A/S |

---

필요시 추가적인 기능이나 다른 삼성 TV 라인업에 대한 정보도 안내해드릴 수 있으니 편하게 문의 주세요!  
부모님께서 만족하실 좋은 TV 선물하시길 바랍니다. 감사합니다! 😊

Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Google Search Query: 삼성전자 2025년 제품 작년 대비 개선점
LLM Query: 삼성전자의 2025년 신제품이 작년 제품과 비교했을 때 어떤 점이 개선되었는지, 주요 성능이나 기능상의 향상 부분을 자세히 알려주세요.
Google Search Query: 삼성전자 2025년 제품 작년 대비 개선점
LLM Query: 삼성전자의 2025년 신제품이 작년 제품과 비교했을 때 어떤 점이 개선되었는지, 주요 성능이나 기능상의 향상 부분을 자세히 알려주세요.
Google Search Results: 3
Google Search Results: 3
Request failed: Redirect response '308 Permanent Redirect' for url 'https://news.zum.com/articles/96015509'
Redirect location: '/articles/96015509/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-%EC%98%AC%ED%95%B4-hbm-%EA%B3%B5%EA%B8%89-%EC%9E%91%EB%85%84-2%EB%B0%B0-%EA%B0%9C%EC%84%A0-hbm3e-3%EC%9B%94%EB%A7%90-%EC%96%91%EC%82%B0?'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/308
Request failed: Redirect response '308 Permanent Redirect' for url 'https://news.zum.com/articles/96015509'
Redirect location: '/articles/96015509/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-%EC%98%AC%ED%95%B4-hbm-%EA%B3%B5%EA%B8%89-%EC%9E%9

...

삼성전자가 소유한 JBL과 하만카돈(Harman Kardon)은 모두 하만 인터내셔널(Harman International) 산하의 오디오 브랜드로, 각각 고유의 특징과 강점을 가지고 있습니다. 두 브랜드의 차이점과 주요 특징, 음향 기술, 제품군, 그리고 적합한 용도에 대해 자세히 설명드리겠습니다.

---

## 1. 브랜드 개요

### JBL
- **특징:** JBL은 강력하고 다이나믹한 사운드로 유명하며, 특히 베이스와 고음역에서 뛰어난 성능을 보여줍니다. 젊고 활동적인 라이프스타일을 겨냥한 제품들이 많아 휴대용 스피커, 스포츠용 이어폰 등에 강점을 가집니다.
- **역사:** 프로페셔널 오디오 장비부터 시작해 대중용 오디오까지 폭넓은 영역을 커버하고 있습니다.
  
### 하만카돈 (Harman Kardon)
- **특징:** 하만카돈은 세련된 디자인과 고급스러운 사운드에 중점을 둔 브랜드로, 정밀한 음향 밸런스와 깔끔한 중저음이 특징입니다. 프리미엄 홈 오디오, AV 시스템, 고급 차량 오디오 등에 주로 사용됩니다.
- **역사:** 1950년대부터 고품질 오디오 기기를 생산해온 전통 있는 브랜드입니다.

---

## 2. 주요 특징 및 음향 기술

| 구분         | JBL                                       | 하만카돈 (Harman Kardon)                      |
|--------------|------------------------------------------|-----------------------------------------------|
| 음향 성향    | 강력한 베이스, 에너지 넘치는 사운드       | 부드럽고 정밀한 사운드, 밸런스가 뛰어남       |
| 타깃 소비자  | 야외 활동, 파티, 스포츠 등 활동적인 라이프스타일 | 고급 홈 시어터, 인테리어와 조화로운 오디오 경험 희망자 |
| 음향 기술    | JBL Signature Sound, 프로급 튜닝, 파티부스트 | 크리스탈 클리어 사운드, 듀얼 하이브리드 드라이버 등 고급 기술 |
| 디자인      | 튼튼하고 활동적인 느낌, 방수 기능 포함한 모델 다수 | 미니멀하고 우아한 디자인, 고급 소재 사용     |

---

## 3. 제품군

### JBL
- 휴대용 블루투스 스피커 (예: JBL Flip, JBL Charge 시리즈)
- 무선 이어폰 및 헤드폰 (예: JBL Free, JBL Live)
- 프로 오디오 스피커 및 PA 시스템
- 자동차 오디오 시스템

### 하만카돈
- 고급 블루투스 스피커 (예: Harman Kardon Onyx Studio)
- 홈 시어터 시스템 및 사운드바
- 프리미엄 자동차 오디오 시스템
- 무선 헤드폰 및 이어폰

---

## 4. 적합한 용도

| 용도               | JBL 추천 이유                                  | 하만카돈 추천 이유                          |
|--------------------|-----------------------------------------------|---------------------------------------------|
| 야외 활동, 여행    | 휴대성 좋고 방수 기능, 강한 베이스로 활기찬 음향 제공 | 상대적으로 휴대용 라인업 적음, 실내용 적합    |
| 홈 오디오          | 적당한 음질과 내구성, 가격 대비 성능 우수        | 고급 사운드와 세련된 디자인으로 프리미엄 홈 오디오에 적합 |
| 자동차 오디오      | 다양한 라인업과 튜닝으로 대중적 선택지             | 하만카돈은 고급 차량에 최적화된 프리미엄 사운드 제공 |
| 전문 음향 장비     | 프로페셔널용 스피커 및 PA 시스템으로 신뢰받음      | 전문 음향보다는 고급 소비자용 중심               |

---

## 5. 요약

- **JBL**은 활동적이고 강렬한 사운드, 휴대성과 내구성에 초점을 맞춰 젊은 층과 아웃도어, 스포츠, 파티 환경에 적합합니다.
- **하만카돈**은 세련되고 정밀한 음향 밸런스, 고급스러운 디자인으로 프리미엄 홈 오디오와 고급 차량용 오디오에 최적화되어 있습니다.

따라서 사용 목적과 선호하는 사운드 스타일에 따라 두 브랜드 중 선택하시면 됩니다.

---

필요하신 제품 추천이나 추가 정보가 필요하시면 언제든 말씀해 주세요!

Original Input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요
Google Search Query: 갤럭시 버즈 이어버드 한쪽 페어링 방법
LLM Query: 갤럭시 버즈 이어버드를 한쪽만 새로 구매했을 때, 새 이어버드를 기존 이어버드와 정상적으로 페어링하는 방법을 자세히 알려주세요.
Google Search Query: 갤럭시 버즈 이어버드 한쪽 페어링 방법
LLM Query: 갤럭시 버즈 이어버드를 한쪽만 새로 구매했을 때, 새 이어버드를 기존 이어버드와 정상적으로 페어링하는 방법을 자세히 알려주세요.
Google Search Results: 3
Google Search Results: 3


갤럭시 버즈 이어버드를 한쪽만 새로 구매하여 기존 이어버드와 정상적으로 페어링하려면 다음 절차를 따라주세요.

---

## 갤럭시 버즈 이어버드 한쪽 새로 구입 후 페어링 방법

### 1. 이어버드 상태 확인 및 준비
- 새로 구입한 이어버드와 기존 이어버드 모두 충전 케이스에 각각 한 쪽씩 정상적으로 안착시킵니다.
- 충전 케이스의 LED가 녹색 또는 빨간색으로 깜빡이는지 확인하세요.
- LED가 깜빡이지 않으면 이어버드 또는 케이스 단자 접점에 이물질이 있는지, 부식 또는 침수 여부를 점검하세요.

### 2. 이어버드 셀프 커플링 진행
- 충전 케이스를 열어 둔 상태에서 케이스 하단 충전 단자 옆에 있는 **연결 버튼을 7초 이상 길게 누릅니다**.
- 이후 충전 케이스 LED가 파란색으로 깜빡이고 나서 꺼지면 커플링이 완료된 상태입니다.
- 만약 LED가 빨간색으로 계속 깜빡이면 1단계부터 다시 시도하세요.

### 3. 모바일 기기와 연결 및 소프트웨어 업데이트
- 양쪽 이어버드를 케이스에 넣고 뚜껑을 닫았다가 다시 열어주세요.
- 스마트폰에서 **Galaxy Wearable(갤럭시 웨어러블) 앱**을 실행해 블루투스로 이어버드를 연결합니다.
- 이어버드 설정 메뉴에서 소프트웨어 업데이트를 진행하면 자동으로 페어링 모드로 진입됩니다.

### 4. 서비스 센터 방문이 필요한 경우
- 기존 이어버드와 새로 구매한 이어버드의 소프트웨어 버전이 다르거나, 셀프 커플링이 어려운 경우는 서비스 센터 방문이 필요합니다.
- 서비스 센터에서 엔지니어가 이어버드 구입 처리 및 양쪽 이어버드 커플링 작업을 진행해 줍니다.
- 배터리 타입이 다른 이어버드끼리는 커플링이 불가능하므로, 2019년 9월 생산분과 이후 생산분의 이어버드는 반드시 같은 배터리 타입인지 확인해야 합니다.

---

### 요약

| 단계                 | 내용                                         |
|--------------------|--------------------------------------------|
| 1단계               | 새 이어버드와 기존 이어버드를 케이스에 넣어 정상 안착 확인          |
| 2단계               | 케이스 열고 연결 버튼 7초 이상 누르기 → LED 파란색 깜빡임 확인       |
| 3단계               | 갤럭시 웨어러블 앱에서 블루투스 연결 및 소프트웨어 업데이트 진행    |
| 4단계 (필요 시)      | 서비스 센터 방문하여 엔지니어에게 커플링 및 이어버드 교체 요청       |

---

필요시 삼성전자 서비스센터 방문을 통해 전문적인 도움을 받으시면 원활한 커플링이 가능합니다.

추가 문의가 있으면 언제든지 알려주세요!

Original Input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나
Google Search Query: 삼성전자 S25 무게 S24 비교 차이
LLM Query: 삼성전자 S25 모델의 무게가 S24 모델과 비교했을 때 얼마나 차이가 나는지 알려주세요. 두 모델의 무게 차이에 대한 구체적인 수치를 알고 싶습니다.
Google Search Query: 삼성전자 S25 무게 S24 비교 차이
LLM Query: 삼성전자 S25 모델의 무게가 S24 모델과 비교했을 때 얼마나 차이가 나는지 알려주세요. 두 모델의 무게 차이에 대한 구체적인 수치를 알고 싶습니다.
Google Search Results: 3
Google Search Results: 3


삼성전자 갤럭시 S25 모델과 S24 모델의 무게 차이는 다음과 같습니다.

- **갤럭시 S25 기본형**  
  - 무게: 162g  
  - 전작 S24 기본형 무게: 167g  
  - **차이: 5g 더 가벼워짐**

- **갤럭시 S25 플러스**  
  - 무게 정보는 구체적으로 제공되지 않았으나, 두께 및 디자인이 얇아지고 가벼워졌다는 점에서 소폭 경량화 예상

- **갤럭시 S25 울트라**  
  - 무게: 218g  
  - 전작 S24 울트라 무게: 232g  
  - **차이: 14g 더 가벼워짐**

요약하면, S25 기본 모델은 약 5g, 울트라 모델은 약 14g 정도 S24보다 가벼워져 전체적으로 슬림하고 경량화된 디자인을 구현했습니다.